In [68]:
import os
import sys
from collections import defaultdict, Counter
from functools import reduce, partial

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import bootstrap

from Bio import SeqIO

In [24]:
mus_cytb = dict()
mus_nd1 = dict()
human_cytb = dict()
human_nd1 = dict()

In [25]:
mus_cytb["aln"] = [str(x.seq) for x in SeqIO.parse("../data/exposure/mus_cytb/alignment_checked.fasta", format="fasta")]
mus_nd1["aln"] = [str(x.seq) for x in SeqIO.parse("../data/exposure/mus_nd1/alignment_checked.fasta", format="fasta")]

human_cytb["aln"] = [str(x.seq) for x in SeqIO.parse("../data/exposure/human_cytb/alignment_checked.fasta", format="fasta")]
human_nd1["aln"] = [str(x.seq) for x in SeqIO.parse("../data/exposure/human_nd1/alignment_checked.fasta", format="fasta")]

In [167]:
def get_consensus(aln):
    seq = []
    for column in zip(*aln):
        nuc = Counter(column).most_common()[0][0]
        seq.append(nuc)
    return "".join(seq)

In [168]:
human_cytb["consensus"] = get_consensus(human_cytb["aln"])
human_nd1["consensus"] = get_consensus(human_nd1["aln"])
mus_cytb["consensus"] = get_consensus(mus_cytb["aln"])
mus_nd1["consensus"] = get_consensus(mus_nd1["aln"])

In [169]:
human_cytb["consensus"]

'ATGACCCCAATACGCAAAACTAACCCCCTAATAAAATTAATTAACCACTCATTCATCGACCTCCCCACCCCATCCAACATCTCCGCATGATGAAACTTCGGCTCACTCCTTGGCGCCTGCCTGATCCTCCAAATCACCACAGGACTATTCCTAGCCATGCACTACTCACCAGACGCCTCAACCGCCTTTTCATCAATCGCCCACATCACTCGAGACGTAAATTATGGCTGAATCATCCGCTACCTTCACGCCAATGGCGCCTCAATATTCTTTATCTGCCTCTTCCTACACATCGGGCGAGGCCTATATTACGGATCATTTCTCTACTCAGAAACCTGAAACATCGGCATTATCCTCCTGCTTGCAACTATAGCAACAGCCTTCATAGGCTATGTCCTCCCGTGAGGCCAAATATCATTCTGAGGGGCCACAGTAATTACAAACTTACTATCCGCCATCCCATACATTGGGACAGACCTAGTTCAATGAATCTGAGGAGGCTACTCAGTAGACAGTCCCACCCTCACACGATTCTTTACCTTTCACTTCATCTTGCCCTTCATTATTGCAGCCCTAGCAGCACTCCACCTCCTATTCTTGCACGAAACGGGATCAAACAACCCCCTAGGAATCACCTCCCATTCCGATAAAATCACCTTCCACCCTTACTACACAATCAAAGACGCCCTCGGCTTACTTCTCTTCCTTCTCTCCTTAATGACATTAACACTATTCTCACCAGACCTCCTAGGCGACCCAGACAATTATACCCTAGCCAACCCCTTAAACACCCCTCCCCACATCAAGCCCGAATGATATTTCCTATTCGCCTACACAATTCTCCGATCCGTCCCTAACAAACTAGGAGGCGTCCTTGCCCTATTACTATCCATCCTCATCCTAGCAATAATCCCCATCCTCCATATATCCAAACAACAAAGCATAATATTTCGCCCACTAAGCCAATCACTTTATTGACTCCTAGCCGCAGACCTCCTC

In [170]:
len(human_cytb["consensus"])

1137

In [34]:
human_cytb["ms_obs192"] = pd.read_csv("../data/exposure/human_cytb/ms/ms192syn_human_cytb.tsv", sep="\t")
human_nd1["ms_obs192"] = pd.read_csv("../data/exposure/human_nd1/ms/ms192syn_human_nd1.tsv", sep="\t")
mus_cytb["ms_obs192"] = pd.read_csv("../data/exposure/mus_cytb/ms/ms192syn_mus_cytb.tsv", sep="\t")
mus_nd1["ms_obs192"] = pd.read_csv("../data/exposure/mus_nd1/ms/ms192syn_mus_nd1.tsv", sep="\t")

In [35]:
human_cytb["ms_exp192"] = pd.read_csv("../data/exposure/human_cytb/pyvolve/out/ms192syn_human_cytb_simulated.tsv", sep="\t")
human_nd1["ms_exp192"] = pd.read_csv("../data/exposure/human_nd1/pyvolve/out/ms192syn_human_nd1_simulated.tsv", sep="\t")
mus_cytb["ms_exp192"] = pd.read_csv("../data/exposure/mus_cytb/pyvolve/out/ms192syn_mus_cytb_simulated.tsv", sep="\t")
mus_nd1["ms_exp192"] = pd.read_csv("../data/exposure/mus_nd1/pyvolve/out/ms192syn_mus_nd1_simulated.tsv", sep="\t")

In [39]:
human_cytb["mutations"] = pd.read_csv("../data/exposure/human_cytb/observed_mutations_iqtree.tsv", sep="\t").sort_values("PosInGene")
human_nd1["mutations"] = pd.read_csv("../data/exposure/human_nd1/observed_mutations_iqtree.tsv", sep="\t").sort_values("PosInGene")
mus_cytb["mutations"] = pd.read_csv("../data/exposure/mus_cytb/observed_mutations_iqtree.tsv", sep="\t").sort_values("PosInGene")
mus_nd1["mutations"] = pd.read_csv("../data/exposure/mus_nd1/observed_mutations_iqtree.tsv", sep="\t").sort_values("PosInGene")

In [126]:
import warnings
warnings.filterwarnings("ignore")

In [134]:
for data in [human_cytb, human_nd1, mus_cytb, mus_nd1]:
    df = data["ms_obs192"].rename(columns={"MutSpec": "MutSpec_obs"}).merge(
        data["ms_exp192"].rename(columns={"MutSpec": "MutSpec_exp"}), on="Mut"
    )
    df["diff"] = df["MutSpec_obs"] - df["MutSpec_exp"]
    diff_stats = df.groupby("Mut")["diff"].agg(
        ["mean", "std", lambda x: bootstrap((x,), np.mean).confidence_interval])
    diff_stats["low"] = diff_stats["<lambda_0>"].apply(lambda x: x.low)
    diff_stats["high"] = diff_stats["<lambda_0>"].apply(lambda x: x.high)

    data["diff_stats"] = diff_stats
    data["muts_positive"] = diff_stats[(diff_stats['low'] > 5e-3) & (diff_stats["std"] > 0)].index.values
    data["muts_negative"] = diff_stats[(diff_stats['high'] < -5e-3) & (diff_stats["std"] > 0)].index.values


In [135]:
for lbl, data in zip(['human_cytb', 'human_nd1', 'mus_cytb', 'mus_nd1'], [human_cytb, human_nd1, mus_cytb, mus_nd1]):
    print(lbl)
    print("muts_positive:", list(data["muts_positive"]))
    print("muts_negative:", list(data["muts_negative"]))
    print()

human_cytb
muts_positive: ['A[G>A]C', 'A[T>C]C', 'C[C>T]C', 'C[G>A]T', 'G[C>T]C', 'G[C>T]T', 'G[T>C]T', 'T[C>T]C', 'T[G>A]G']
muts_negative: ['A[A>G]A', 'A[G>A]A', 'A[G>A]T', 'C[A>G]A', 'C[C>T]G', 'C[G>A]A', 'C[T>C]G', 'G[A>G]A', 'G[C>T]A', 'G[T>C]A', 'T[A>G]A', 'T[G>A]A', 'T[G>A]T', 'T[T>C]A']

human_nd1
muts_positive: ['C[A>G]C', 'C[C>T]C', 'G[C>T]C', 'T[A>G]T', 'T[C>A]G']
muts_negative: ['A[A>G]T', 'A[C>T]A', 'A[C>T]C', 'A[G>A]A', 'C[A>G]A', 'C[C>T]A', 'G[A>G]A', 'G[A>G]G', 'G[C>T]T', 'G[T>C]A', 'T[A>G]G', 'T[C>T]A']

mus_cytb
muts_positive: ['A[T>C]C', 'A[T>C]T', 'G[A>G]C', 'G[A>G]G', 'G[A>T]C', 'T[A>G]G', 'T[G>A]C', 'T[G>A]G', 'T[G>A]T']
muts_negative: ['A[A>G]A', 'A[C>T]A', 'A[C>T]C', 'C[A>G]C', 'C[C>T]A', 'C[C>T]G', 'G[C>T]C', 'T[A>G]C', 'T[A>G]T', 'T[A>T]C', 'T[C>T]T']

mus_nd1
muts_positive: ['A[C>T]C', 'A[T>C]C', 'C[A>G]G', 'C[A>G]T', 'C[G>A]C', 'G[A>G]A', 'G[A>G]C', 'G[C>T]T', 'G[G>A]C', 'G[G>A]T', 'G[T>C]T', 'T[A>G]C', 'T[C>T]C', 'T[C>T]T', 'T[G>A]T']
muts_negative: ['A[A>G

In [187]:
fout = open("../data/exposure/consensus_labels.txt", "w")
for lbl, data in zip(['human_cytb', 'human_nd1', 'mus_cytb', 'mus_nd1'], [human_cytb, human_nd1, mus_cytb, mus_nd1]):
    fout.write(">"+lbl+"\n")
    fout.write(data['consensus']+"\n")
    muts_plus = data["mutations"][data["mutations"].Mut.isin(data["muts_positive"])]
    muts_minus = data["mutations"][data["mutations"].Mut.isin(data["muts_negative"])]

    muts_plus_est = muts_plus.groupby(["Mut", "PosInGene"]).ProbaFull.sum()
    muts_minus_est = muts_minus.groupby(["Mut", "PosInGene"]).ProbaFull.sum()

    pos_plus  = set(muts_plus_est[muts_plus_est > 0.5].reset_index().PosInGene.values)
    pos_minus = set(muts_minus_est[muts_minus_est > 0.5].reset_index().PosInGene.values)

    for pos in range(1, len(data['consensus'])+1):
        if pos in pos_plus:
            fout.write("+")
        elif pos in pos_minus:
            fout.write("-")
        else:
            fout.write(" ")
    fout.write("\n\n")

fout.close()

In [188]:
fout = open("../data/exposure/consensus_labels_syn.txt", "w")
for lbl, data in zip(['human_cytb', 'human_nd1', 'mus_cytb', 'mus_nd1'], [human_cytb, human_nd1, mus_cytb, mus_nd1]):
    fout.write(">"+lbl+"\n")
    fout.write(data['consensus']+"\n")
    muts_plus = data["mutations"][data["mutations"].Mut.isin(data["muts_positive"])]
    muts_minus = data["mutations"][data["mutations"].Mut.isin(data["muts_negative"])]

    muts_plus_est = muts_plus[muts_plus.Label >= 1].groupby(["Mut", "PosInGene"]).ProbaFull.sum()
    muts_minus_est = muts_minus[muts_minus.Label >= 1].groupby(["Mut", "PosInGene"]).ProbaFull.sum()

    pos_plus  = set(muts_plus_est[muts_plus_est > 0.5].reset_index().PosInGene.values)
    pos_minus = set(muts_minus_est[muts_minus_est > 0.5].reset_index().PosInGene.values)

    for pos in range(1, len(data['consensus'])+1):
        if pos in pos_plus:
            fout.write("+")
        elif pos in pos_minus:
            fout.write("-")
        else:
            fout.write(" ")
    fout.write("\n\n")

fout.close()

In [179]:
muts_plus[muts_plus.ProbaMut > 0.5]

,Mut,Label,PosInGene,PosInCodon,RefCodon,AltCodon,RefAa,AltAa,ProbaRef,ProbaMut,ProbaFull,RefNode,AltNode,Gene
6509,T[C>T]C,1,12,3,TTC,TTT,F,F,0.637472,0.541049,0.491054,Node52,Node53,1
1847,T[C>T]C,1,12,3,TTC,TTT,F,F,0.815786,0.815786,0.673469,Node1,RN_129,1
5717,T[C>T]C,1,12,3,TTC,TTT,F,F,0.691573,0.691573,0.626503,Node51,RN_130,1
3184,T[C>T]T,1,36,3,ATC,ATT,I,I,0.672148,0.672148,0.575858,Node2,RN_117,1
8091,G[G>A]T,2,42,3,GGG,GGA,G,G,1.000000,1.000000,0.999137,Node36,RN_53,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7350,G[A>G]C,1,846,3,TGA,TGG,W,W,0.983618,0.983618,0.892729,Node52,RN_115,1
7355,T[C>T]C,1,867,3,TTC,TTT,F,F,0.582518,0.582518,0.528691,Node52,RN_115,1
8071,G[G>A]T,0,877,1,GTA,ATA,V,M,0.999050,0.999050,0.998188,Node38,RN_76,1
8018,G[G>A]T,0,877,1,GTA,ATA,V,M,0.987250,0.987240,0.986459,Node20,Node40,1


In [186]:
a = muts_plus.groupby(["Mut", "PosInGene"]).ProbaFull.sum()
a[a > 0.5].reset_index().PosInGene

0     182
1     519
2     768
3     786
4     519
     ... 
57    663
58    699
59    748
60    823
61    843
Name: PosInGene, Length: 62, dtype: int64